# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: Victor Sznifer Leibovicius e Alessandra Blucher.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: project_twocd


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)--   g4gITvNPFN5S2WJ8FseSKRah8
    1. Consumer Secret (API Secret) -- InRrTZALEMZ9owMSh9BH6mdiGJxywJQ4kKlOlvCQnU7ZMcLfkW
1. Mais abaixo, gere um Token e anote também:
    1. Access Token --  905123126941536257-W88x7exx7pDDM4NGM9DyYVJbKqGvFTH
    1. Access Token Secret --   4fsABI2ri7rhNvsUdy3HK8kJrfpgtdJQ7IQuj7oybnWN9
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @project_twocd

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'McDonalds'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
excel = pd.read_excel('McDonalds(1).xlsx')
total = pd.DataFrame(excel.loc[:, ["Treinamento", "Relevância"]])


relevantes = excel[excel.Relevância=="R"]
irrelevantes = excel[excel.Relevância=="I"]
total.head(5)

,Treinamento,Relevância
0,"i'm at @mcdonalds_br in taboão da serra, sp ht...",I
1,rt @mcdonalds_br: um encontro com muita coisa ...,R
2,funcionário do mcdonalds fazem greve inédita n...,I
3,rt @vejasp: mcdonald’s lança duas novas sobrem...,I
4,@mcdonalds_br fazer uns bonequinhos do exo com...,I


In [8]:
palavras_total = []
palavras_total_sem_repetidas = []
palavras_total_certo = []                               #Esse será o total usado para o cálculo da probabilidade
superfulos = [',',';','.','!','?',"(",")", ':', '"', "'"]

In [9]:
#Filtragem do total de palavras

for tweet in total.Treinamento:
    a = tweet.split()
    for palavra in a:
        if palavra not in superfulos:
            palavras_total.append(palavra)

for p in palavras_total:                                    #Teste com e sem repetidas
    if p not in palavras_total_sem_repetidas:
        palavras_total_sem_repetidas.append(p)

for q in range(len(palavras_total_sem_repetidas)):
    variavel = ''.join(palavras_total_sem_repetidas[q])
    if len(variavel)!=0:
        if variavel[0] !='@' and variavel!= 'rt' and variavel[0:5]!='https':
            palavras_total_certo.append(variavel)
                    
num_total=len(palavras_total_certo)
print(num_total)

1084


In [10]:
# Separação entre Relevantes e Irrelevantes levando em conta somente as palavras filtradas
palavras_relevantes = []
for tweet in relevantes.Treinamento:
    t = tweet.split()
    for palavra in t:
        if palavra in palavras_total_certo:
            palavras_relevantes.append(palavra)
            
num_relevantes = len(palavras_relevantes)
            
palavras_irrelevantes = []
for tweet in irrelevantes.Treinamento:
    t = tweet.split()
    for palavra in t:
        if palavra in palavras_total_certo:
            palavras_irrelevantes.append(palavra)

            
num_irrelevantes = len(palavras_irrelevantes)

In [11]:
#Função que calcula as probabilidades
def naive_bayes(tweet):
    lista_prob_r = []
    lista_prob_i = []
    prob_r_final = prob_i_final= 1
    r = i = 0
    t = tweet.split()
    
    
    for palavra in t:
        if palavra in palavras_relevantes:
            r+=1
        prob_r = (1+r)/(num_total+num_relevantes)
        lista_prob_r.append(prob_r)
    for z in range(len(lista_prob_r)):
        prob_r_final *= lista_prob_r[z] 
    
    
    for palavra in t:
        if palavra in palavras_irrelevantes:
            i+=1
        prob_i = (1+i)/(num_total+num_irrelevantes)
        lista_prob_i.append(prob_i)
    for w in range(len(lista_prob_i)):
        prob_i_final *= lista_prob_i[w]
        
    if prob_r_final > prob_i_final:
        return 'R'
    if prob_r_final <= prob_i_final:
        return 'I'
    

In [12]:
naive_bayes("hey @mcdonalds_br quero um cuponzinho de top sunday por 5 reais ☹")

'R'

In [13]:
# RANGE DE RELEVANCIA A FUNCAO DEVE DEVOLVER A MAXIMA PROBABILIDADE E MINIMA

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [14]:
# ABRINDO O EXCEL DE TESTE PARA TESTAR O CLASSIFICADOR
Teste = pd.read_excel('Teste.xlsx')

# CRIANDO UM DATAFRAME SÓ COM OS TWEETS E OUTRO DATAFRAME SÓ COM A RELEVÂNCIA

teste_tweets=pd.DataFrame(Teste.loc[:,["Teste"]])
teste_relev=pd.DataFrame(Teste.loc[:,["Relevância"]])

In [17]:
def performance():
    relevância = []
    tweets = []
    positivo_verdadeiro = positivo_falso = negativo_verdadeiro = negativo_falso = 0
    for x in teste_relev.Relevância:
        relevância.append(x)
    for y in teste_tweets.Teste:
        nb = naive_bayes(y)
        print(nb)
        tweets.append(nb)

    for z in range(len(relevância)):
        if relevância[z] == 'R':
            if tweets[z] == 'R':
                positivo_verdadeiro += 1
            if tweets[z] == 'I':
                negativo_falso += 1
        if relevância[z] == 'I':
            if tweets[z] == 'R':
                positivo_falso += 1
            if tweets[z] == 'I':
                negativo_verdadeiro += 1
                
    probabilidade_pv = ((positivo_verdadeiro/200)*100)
    probabilidade_pf = ((positivo_falso/200)*100)
    probabilidade_nv = ((negativo_verdadeiro/200)*100)
    probabilidade_nf = ((negativo_falso/200)*100)
    
    acertos = probabilidade_pv + probabilidade_nv
    erros = probabilidade_pf + probabilidade_nf
    return (acertos,erros,probabilidade_pv,probabilidade_nv,probabilidade_pf,probabilidade_nf)
    

In [18]:
# RODANDO A FUNÇÃO ACERTOS

performance()

R
I
R
R
R
R
R
R
I
I
R
I
R
R
R
R
R
R
R
R
I
R
I
R
R
R
R
I
R
R
R
R
I
R
R
R
R
R
R
R
R
I
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
I
R
R
R
I
R
R
I
R
R
R
I
R
R
R
R
R
R
R
R
R
I
R
R
R
R
R
R
R
R
I
R
R
R
R
R
R
R
R
R
I
R
I
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
I
R
R
R
R
R
R
R
R
I
R
R
R
R
R
R
R
R
I
R
R
R
I
I
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
I
I
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
I
R
R
R


(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
